In [6]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup as bs
import time

In [7]:
# 출처 : https://www.youtube.com/watch?app=desktop&v=pj4GCSsIxSk

In [8]:
driver = wb.Chrome()
driver.get("https://map.kakao.com/")

In [9]:
# 1. 검색창의 위치를 알아오기
# 2. 광주맛집이라는 키워드를 입력
# 3. 검색버튼 실행

# 우측 상단 파란 이미지 클릭해주는 코드
# → 이렇게 해야 접근이 가능
img = driver.find_element(By.CSS_SELECTOR,"div.inner_coach_layer")
img.click()

In [10]:
# 카카오맵 검색창 id 가져온다
# id=search.keyword.query를 순수하게 문자열로 가져오기 위해 \ <- 넣어준다
# 선택자 . 키워드는 클래스값을 명시하는 방법
# "." 문자 점으로 인식하고 싶을 때는 \을 넣어주면 해결된다

# 현재 search에는 검색창의 위치가 담겨있다
search = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.query")

In [11]:
# search에 값을 보내겠다 (send_keys) -> 검색창에 값 들어감
search.send_keys("광주맛집")

In [12]:
# 검색 버튼
searchBtn = driver.find_element(By.CSS_SELECTOR, "#search\.keyword\.submit")
searchBtn.click()

In [13]:
# 드라이버에서 바로 find하지 않고, BeautifulSoup 객체화를 진행한 다음 데이터를 수집한다
# - 내가 화면에서 보고 있는 데이터를 BeautifulSoup 객체화

# BeautifulSoup 만들때 request에서 받아왔던 requset.text 정보, lxml이라는 파싱 방법 통해 문자 데이터를 컴퓨터가 알 수 있는 html 언어로 BeautifulSoup 객체화 진행했었다
# 그런데 request 통해 가져온 데이터 없지 않음? -> 맞다
# 현재 우리가 웹드라이버 통해 브라우저가 정보를 담고 있는 것이기 때문에 request 객체에 있는 데이터는 존재할 수 없다

# 그러면 bs() 안에 브라우저가 갖고 있는 페이지 정보를 넣어줘야 한다
# 이 브라우저가 뭐냐면 driver (현재 내가 보고 있는 지도 화면의 정보를 알고 있는 것이 driver 객체)
# BeautifulSoup과 Selenium 합쳐서 데이터를 가져온다

soup = bs(driver.page_source, "lxml")

In [14]:
# soup에서 데이터 가져오는 명령 soup.select
soup.select

<bound method Tag.select of <html class="index" lang="ko"><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1024" name="viewport"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_A.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_B.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_C.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_D.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/canvas_roadview.css" rel="stylesheet"/>
<link href="https://t1.daumcdn.net/kakaomapweb/subway/linemap/canvas/prod/css/subway.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<title>카카오맵</title>
<script type="text/javascript">
if (false === true && true && document.referrer.split('?')[0] === "https://accounts.kakao.com/login?continue=".split('?')[0]) {
	window.location.href 

In [15]:
# 가게 이름은 a 태그의 class="link_name"
# 광주의 음식점 나온다
title = soup.select("a.link_name")

In [16]:
# 내가 필요한 것은 순수한 글자 정보
# 넘겨받은 타입 list, list는 크기가 정해져 있는 데이터라서 반복문 for 활용

for i in title:
    # 글자정보만 print
    print(i.text)

광주맛집
광주맛집 소신식당
광주휴대폰맛집
광주한우맛집 소신정육점
광주맛집효심싸닭갈비
광주누수맛집 라이동합설비
광주집
거진항맛집코다리네 광주본점
광주무등산
광주국밥
광주수산
진짜광주식당
광주식당
전통맛집할매순대국 광주초월점
광주똑순이아구찜


In [17]:
# 장소 더보기 버튼 클릭
# 장소 더보기 검사 - a 태그의 id="info.search.place.more"
btnMore = driver.find_element(By.CSS_SELECTOR, "#info\.search\.place\.more")
btnMore.click()
# 누르면 더보기 버튼 누르면서 한 페이지 넘어간다

In [19]:
# 이제 페이지가 1, 2, 3 ... 인지 번호 알아와서 데이터 클릭한 다음 수집 진행하는 반복문 코드 짜야한다
# 내가 몇 페이지 클릭하고 있는지 번호 값을 알아와야 함
# 페이지 번호 2에 우클릭 -> 검사
# 각각의 a 태그를 통해 페이지를 바꾸고 있다
# 우리에게 필요한 것은 1 ~ 5페이지까지 클릭할 수 있는 5개의 a 태그이다

# id는 중복 안되어서 전부 다르고, class는 INACTIVE, ACTIVE 클릭할 때마다 값이 바뀐다
# -> id와 class 값으로는 데이터를 한번에 불어오기 어렵다
# -> 이럴 때는 부모 태그를 본다

# a 태그의 부모 태그를 보면, div 태그 존재하고 class값으로 pageWrap 존재
# 그리고 이것의 부모도 마찬가지로 id값 갖고 있다 (info.search.page)

# 페이지 번호 데이터 수집
# 첫 번째 부모님은 div 태그, id="info.search.page"
# 그 밑에 있는 div를 갖고온 다음 그 밑의 a 태그를 불러오겠다고 접근하면 5개의 a 태그 한번에 불러올 수 있다

page = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page > div > a")

In [28]:
# 첫 번째 페이지 클릭한 다음, 글자 정보 수집하고 끝나면 두 번째 페이지 클릭해서 글자 수집
# 이렇게 클릭 - 들어가서 글자수집 반복

try:
    while True:
        for i in range(6):

            # 페이지 5번까지 갔으면 카카오맵의 ">" 클릭해줘야 한다
            # 조건 줘야한다. 만약 내가 클릭한 데이터가 5보다 크다면, 다음 버튼을 클릭해랴는 명령이 필요하다

            if(i < 5):

                # 5페이지 까지의 데이터 수집
            
                # page에 있는 i번째 a 태그 가져와서 click
                page[i].click()

                # 페이지가 바뀔 때는 time.sleep을 넣어서 페이지가 쉴 수 있고 데이터를 업로드하고 로딩할 수 있는 시간 벌어야 한다
                time.sleep(2)

                # 현재 페이지에 있는 정보를 새로 받아야 한다 -> soup 데이터로 바꿔줘야 한다 (데이터 바꿔주는 BeautifulSoup 코드)
                # 화면이 바뀌었기 때문에 최신화된 화면의 정보가 필요함 -> 매번 클릭할 때마다 새로운 soup 데이터 필요하다
                soup = bs(driver.page_source, "lxml")

                # 데이터 출력하는 문장이 필요하다
                title = soup.select("a.link_name")

                # 수집한 데이터를 출력
                for i in title:
                    print(i.text)
            
            else:
                #마지막 페이지 클릭했을 때 다음 버튼 클릭하는 로직
                # ">" : id값으로 info.search.page.next를 가진 button 태그
                # more = driver.find_element(By.CSS_SELECTOR, "info\.search\.page\.next")
                more = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
                more.click()
                time.sleep(2)
except:
    print("수집이 완료되었습니다!")

광주맛집
광주맛집 소신식당
광주휴대폰맛집
광주한우맛집 소신정육점
광주맛집효심싸닭갈비
광주누수맛집 라이동합설비
광주집
거진항맛집코다리네 광주본점
광주무등산
광주국밥
광주수산
진짜광주식당
광주식당
전통맛집할매순대국 광주초월점
광주똑순이아구찜
정가득광주본가
광주집
광주식당
광주찌개전문
광주곱창
광주집
카페 광주미용실
광주족발 서대문점
광주식당
광주식당
광주집순대국
광주집 보신탕
광주육전
광주곱창
광주식당
광주식당
광주집
광주회관
광주식당
광주식당
광주집
광주회집
이모네광주곱창
광주식당
광주 섬진강 논현본점
광주회관
청량리광주냉면
한우리광주집
광주식당
광주식당감자탕
광주먹자집
광주식당
광주홍어회찜
광주생오리돌구이전문
광주식당
광주식당
광주식당
광주가든
광주등심 (휴업중)
광주식당
광주한양식당
광주수산
광주집
군산광주회집
신은정광주소머리국밥
광주할머니곱창
가마솥광주통닭
광주사철탕삼계탕
광주집
광주식당
광주식당
광주식당
광주식당
광주함평호남횟집
광주진국밥
광주식당
안양중앙시장곱창골목광주집
광주식당
광주오리탕
광주오리탕
광주식당
광주장성오리탕
광주대명식당
광주식당
광주아구찜.탕
광주생오리돌구이 감일점
광주가든
광주나들목포차
광주식당
광주함평호남회집
광주식당
광주식당
광주신안월미회집
안양명물최고의맛광주집
전라도광주밥상 광주무등횟집
자성화맛집코다리네 광주점
광주머리고기집
광주집
신호남광주회집
내고향광주식당
광주집
광주식당
광주황제닭갈비
광주홍탁
광주소머리국밥
광주포차
광주분식
광주실내포차
광주뚝배기
광주칼국수 경기광주본점
남도맛집
광주공원
태전동맛집코다리네
음식의명가 먹거리마당맛집
광주막 본점
광주정육점식당
광주집
광주아구찜 북어해장국
빛고을광주식당
광주원조감자탕
광주식당
콩카페 경기광주 태전점
광주명가족발
오포자성화맛집코다리네
광주가마솥닭강정
남도맛집
광주공원
태전동맛집코다리네
음식의명가 먹거리마당맛집
광주막 본점
광주정육점식당
광주집
광주아구찜 북어해장국
빛고을광주식당
광주원조감자탕
광주식당
콩카페 경기광주 태전점
광주명가족발
오포자성화맛집코다리네
광주가마솥닭강정
광주초계

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF69086F922+3045234]
	(No symbol) [0x00007FF690513652]
	(No symbol) [0x00007FF6903D7393]
	(No symbol) [0x00007FF690427292]
	(No symbol) [0x00007FF690419FDB]
	(No symbol) [0x00007FF690449DAA]
	(No symbol) [0x00007FF690419916]
	(No symbol) [0x00007FF690449FC0]
	(No symbol) [0x00007FF6904691C4]
	(No symbol) [0x00007FF690449B53]
	(No symbol) [0x00007FF69041813C]
	(No symbol) [0x00007FF690418D71]
	GetHandleVerifier [0x00007FF69089E89D+3237613]
	GetHandleVerifier [0x00007FF6908E0BA7+3508727]
	GetHandleVerifier [0x00007FF6908D976F+3478975]
	GetHandleVerifier [0x00007FF69064F1C6+815126]
	(No symbol) [0x00007FF69051ED0F]
	(No symbol) [0x00007FF690519854]
	(No symbol) [0x00007FF6905199E2]
	(No symbol) [0x00007FF69050A574]
	BaseThreadInitThunk [0x00007FF99EF07344+20]
	RtlUserThreadStart [0x00007FF9A06426B1+33]


In [ ]:
# 페이지 5번까지 갔으면 카카오맵의 ">" 클릭해줘야 한다
# 조건 줘야한다. 만약 내가 클릭한 데이터가 5보다 크다면, 다음 버튼을 클릭해랴는 명령이 필요하다

In [ ]:
# 출처 : https://coding-230227.tistory.com/70

# BeautifulSoup + Selenium을 같이 조화롭게 코드를 작성
# 왜?
    # Selenium은 브라우저 제어 등 자동화에 아주 특화
    # but Selenium 자체로 html 파싱, 데이터 추출에 어느 정도 한계가 있다. 
    # 그렇기 때문에 각자의 장점을 살려서, 짬뽕
    
    
    
# 파싱 (Parsing)이란?
# - 웹페이지에서 원하는 데이터를 추출, 가공하기 쉬운 상태로 변환하는 것
# - 예를 들면, 생선을 낚았다 최종적으로 통조림의 형태로 만들어야 하는데, 손질 하려는 절차를 파싱이라고 한다.

# 생선과는 달리, 웹페이지에 존재하는 데이터는 list, dictionary와 같은 자료구조와 다르다. 

# 그래서 parser라는 함수나 프로그램으로 다루기 쉬운 형태로 바꾸어 주는데

# 이 과정을 '파싱'이라고 한다.

In [1]:
# # 라이브러리 import 

# from selenium import webdriver as wb
# from bs4 import BeautifulSoup as bs

# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys

# import time 
# from tqdm import tqdm
# import pandas as pd

In [2]:
# # 브라우저 할당 
# driver = wb.Chrome()
# driver.get('https://map.kakao.com/')
# driver.maximize_window()

In [3]:
# # 파란색 팝업 창 기억나는가?
#     # 이 부분을 사용자가 클릭해서 없애듯, 
#     # 컴퓨터에게도 똑같이 시켜줘야 한다.
    
# click_blue_popup = driver.find_element(By.CSS_SELECTOR,'body > div.coach_layer.coach_layer_type1 > div > div > div > span')
# click_blue_popup.click()

In [4]:
# # 검색창 지정 (마우스로 올려놓기)

# search = driver.find_element(By.CSS_SELECTOR, '#search\.keyword\.query')

# # 검색어 입력('이수역 맛집')

# search.send_keys('이수역 맛집')

# # ENTER키 누르고 데이터가 로드할 수 있는 시간까지 할당해주기

# search.send_keys(Keys.ENTER)
# time.sleep(3)

In [29]:
# BeautifulSoup으로 객체화

In [5]:
# # 보다 더 빠른 데이터 수집을 위해 -> 추출하는 시간도 이제는 고려

# soup = bs(driver.page_source,'lxml')
# soup

<html class="index" lang="ko"><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1024" name="viewport"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_A.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_B.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_C.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/app_D.css" rel="stylesheet"/>
<link href="//t1.daumcdn.net/kakaomapweb/map/202404091402526/canvas_roadview.css" rel="stylesheet"/>
<link href="https://t1.daumcdn.net/kakaomapweb/subway/linemap/canvas/prod/css/subway.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<title>카카오맵</title>
<script type="text/javascript">
if (false === true && true && document.referrer.split('?')[0] === "https://accounts.kakao.com/login?continue=".split('?')[0]) {
	window.location.href = window.location.origin + '

In [6]:
# # 식당이름 (titles)

# titles = soup.select('a.link_name')

# for i in titles:
#     print(i.text)

애플하우스 이수
그릴진
세녹
소담촌 사당직영점
전주전집
스시로로
방배김밥
호요
원조쯔왕돈까스
남미플랜트랩
단아한정식
신사펍 이수점
낙성곱창
삼육가 사당역1호점
유정우함흥냉면


In [ ]:
# # 식당 주소 -> addresses

# addresses = soup.select('div.addr > p:nth-child(1)')

# for i in addresses:
#     print(i.text)

In [ ]:
# # 더보기 버튼 지정 및 클릭 
#     # 기능적인 부분이니까 XPATH -> 동적인 요소 selenium 사용

# click_more_location = driver.find_element(By.XPATH,'//*[@id="info.search.place.more"]')
# click_more_location.click()
# time.sleep(2)

In [ ]:
# # 더보기 버튼을 누른 후에는 , 번호 버튼들이 존재
#     # 해당 항목들을 수집
# page = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.page > div > a')
# page

In [ ]:
# 크롤러 조각하기

In [ ]:
# # 빈 리스트 2개를 생성 


# titles_list = []
# addresses_list = []

# try : # 시도해봐! 뭘? 아래 실행코드들을
#     while True: # 조건이 참일 경우를 
#         for i in tqdm(range(6)):
#             # 만약 순차가 5 미만일 경우 
#             if i < 5:
#                 # page 클릭
#                 # page = driver.find_elements(By.CSS_SELECTOR,'#info\.search\.page > div > a')
#                     # find_elements 는 요소를 지정해주는 기능이지 값을 저장하는 함수가 아니다! 그래서 for문 반복문에 넣을 필요없음
#                 page[i].click()
#                 time.sleep(2)
                
#                 # beautifulsoup 객체화
#                 soup = bs(driver.page_source,'lxml')
                
#                 # 식당이름 
#                     # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
#                 titles = soup.select('a.link_name')
                
#                 # for문을 돌면서 titles에서 순차적으로 
#                 for i in titles :
#                     # titles_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
#                     titles_list.append(i.text)
                
#                 # 식당주소
#                     # 변수 = soup.select 선택한 데이터 값을 저장하는 함수! 그래서 페이지가 바뀔 때마다 새로 선언해줘야함
#                 addresses = soup.select('div.addr > p:nth-child(1)')
                
#                 # for문을 돌면서 addresses에서 순차적으로 
#                 for i in addresses :
#                     # addresses_list에 순차적으로 뽑은 요소의 텍스트만 넣어줘
#                     addresses_list.append(i.text)
#             else:
#                 # 그렇지 않은 경우는,
#                 # i가 5보다 큰 경우
#                 # 5번 버튼까지 다 돌고, 다음 버튼을 눌러야하는 경우
#                 click_next = driver.find_element(By.XPATH,'//*[@id="info.search.page.next"]')
#                 click_next.click()
#                 time.sleep(3)
# except:
#     print("수집이 완료되었습니다.")

# driver.quit()
# len(titles_list1), len(addresses_list1)